In [2]:
ord("👋")

128075

In [5]:
list("Eddie".encode("utf-16"))

[255, 254, 69, 0, 100, 0, 100, 0, 105, 0, 101, 0]

In [15]:
text = "斜め七十七度の並びで泣く泣く嘶くナナハン七台難なく並べて長眺め Naname nanajyuunana-do no narabi de nakunaku inanaku nanahan nanadai nannaku narabete naganagame."
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

print(tokens)

[230, 150, 156, 227, 130, 129, 228, 184, 131, 229, 141, 129, 228, 184, 131, 229, 186, 166, 227, 129, 174, 228, 184, 166, 227, 129, 179, 227, 129, 167, 230, 179, 163, 227, 129, 143, 230, 179, 163, 227, 129, 143, 229, 152, 182, 227, 129, 143, 227, 131, 138, 227, 131, 138, 227, 131, 143, 227, 131, 179, 228, 184, 131, 229, 143, 176, 233, 155, 163, 227, 129, 170, 227, 129, 143, 228, 184, 166, 227, 129, 185, 227, 129, 166, 233, 149, 183, 231, 156, 186, 227, 130, 129, 32, 78, 97, 110, 97, 109, 101, 32, 110, 97, 110, 97, 106, 121, 117, 117, 110, 97, 110, 97, 45, 100, 111, 32, 110, 111, 32, 110, 97, 114, 97, 98, 105, 32, 100, 101, 32, 110, 97, 107, 117, 110, 97, 107, 117, 32, 105, 110, 97, 110, 97, 107, 117, 32, 110, 97, 110, 97, 104, 97, 110, 32, 110, 97, 110, 97, 100, 97, 105, 32, 110, 97, 110, 110, 97, 107, 117, 32, 110, 97, 114, 97, 98, 101, 116, 101, 32, 110, 97, 103, 97, 110, 97, 103, 97, 109, 101, 46]


In [28]:
def get_stats(tokens):
    counts = {}

    for pair in zip(tokens, tokens[1:]):
        counts[pair] = counts.get(pair, 0) + 1

    return counts
    

In [30]:
counts = get_stats(tokens)

top_pair = max(counts, key=counts.get)
top_pair

(110, 97)

In [33]:
def merge(pair, new_token, tokens):
    new_tokens = []

    i = 0
    while i < len(tokens):
        if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == pair:
            new_tokens.append(new_token)
            i += 2
        else:
            new_tokens.append(tokens[i])
            i += 1

    return new_tokens

In [35]:
merge((2, 3), 5, [1, 2, 3, 4])

[1, 5, 4]

In [36]:
print(len(tokens))

tokens2 = merge(top_pair, 256, tokens)
print(tokens2)
print(len(tokens2))

191
[230, 150, 156, 227, 130, 129, 228, 184, 131, 229, 141, 129, 228, 184, 131, 229, 186, 166, 227, 129, 174, 228, 184, 166, 227, 129, 179, 227, 129, 167, 230, 179, 163, 227, 129, 143, 230, 179, 163, 227, 129, 143, 229, 152, 182, 227, 129, 143, 227, 131, 138, 227, 131, 138, 227, 131, 143, 227, 131, 179, 228, 184, 131, 229, 143, 176, 233, 155, 163, 227, 129, 170, 227, 129, 143, 228, 184, 166, 227, 129, 185, 227, 129, 166, 233, 149, 183, 231, 156, 186, 227, 130, 129, 32, 78, 97, 256, 109, 101, 32, 256, 256, 106, 121, 117, 117, 256, 256, 45, 100, 111, 32, 110, 111, 32, 256, 114, 97, 98, 105, 32, 100, 101, 32, 256, 107, 117, 256, 107, 117, 32, 105, 256, 256, 107, 117, 32, 256, 256, 104, 97, 110, 32, 256, 256, 100, 97, 105, 32, 256, 110, 256, 107, 117, 32, 256, 114, 97, 98, 101, 116, 101, 32, 256, 103, 97, 256, 103, 97, 109, 101, 46]
172
